# 1. 비디오 불러와서 프레임 추출하기

In [1]:
import os
os.getcwd()

'/home/yyj/sahi/demo'

In [2]:
import cv2

def extract_frame(video_path, frame_dir, overwrite=False, start=-1, end=-1, every=1):
    """
    Extract frames from a video using OpenCVs VideoCapture
    :param video_path: path of the video
    :param frames_dir: the directory to save the frames
    :param overwrite: to overwrite frames that already exist?
    :param start: start frame
    :param end: end frame
    :param every: frame spacing
    :return: count of images saved
    """
    
    video_path = os.path.normpath(video_path)
    frame_dir = os.path.normpath(frame_dir)
    
    video_dir, video_filename = os.path.split(video_path)
    video_filename = os.path.splitext(video_filename)[0]
    assert os.path.exists(video_path)
    
    capture = cv2.VideoCapture(video_path)
    
    if start < 0:
        start = 0
    if end < 0:
        end = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    
    fps = capture.get(cv2.CAP_PROP_FPS)
    print(fps)
    
    capture.set(cv2.CAP_PROP_FPS,30) 
    print(capture.get(cv2.CAP_PROP_FPS))
    
    print(start, end)
    capture.set(1, start)
    frame = start
    while_safety = 0
    saved_count = 0 
    
    while frame < end:
        if while_safety > 10:  # break the while if our safety maxs out at 10
            break
        
        _, image = capture.read()
        
        if frame % every == 0:
           
            if image is None:
                print("Image is None")
                while_safety += 1
                continue
            
            while_safety = 0
            
            save_path = os.path.join(frame_dir, video_filename, "{:010d}.jpg".format(saved_count))
            
            if not os.path.exists(os.path.join(frame_dir, video_filename)):
                os.makedirs(os.path.join(frame_dir, video_filename))
            if not os.path.exists(save_path) or overwrite:
                cv2.imwrite(save_path, image)
                saved_count += 1
                
        frame += 1
        
    capture.release()
        
    return saved_count, os.path.join(frame_dir, video_filename);

In [17]:
video_path = "../resources/FL_221024_3.mp4"
frame_dirs = "../resources/export_frame"

_, source_image_dir = extract_frame(video_path, frame_dirs, every=30, overwrite=True)

30.467068557812325
30.467068557812325
0 31168


[h264 @ 0x55d63f40a500] decode_slice_header error


Image is None
Image is None
Image is None
Image is None
Image is None
Image is None
Image is None
Image is None
Image is None
Image is None
Image is None


In [ ]:
source_image_dir = "../resources/FLL_VAL/images/"
source_label_dir = "../resources/FLL_VAL/labels/"

In [5]:
source_image_dir

'../resources/export_frame/FL_221024'

# 2. 비디오 인퍼런스

In [6]:
# arrange an instance segmentation model for test
from sahi.utils.yolov5 import (
    download_yolov5s6_model,
)

# import required functions, classes
from sahi import AutoDetectionModel
from sahi.utils.cv import read_image
from sahi.utils.file import download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict
from sahi.scripts.coco_error_analysis import analyse
from sahi.scripts.coco_evaluation import evaluate
from sahi.slicing import slice_image
from IPython.display import Image
from pathlib import Path
import json
import os
import cv2
import shutil
from glob import glob
from pathlib import Path
from PIL import Image

In [7]:
fll_model_221023_path = '../resources/models/221023_960/best.pt'

In [8]:
fll_model_221023 = AutoDetectionModel.from_pretrained(
    model_type='yolov5',
    model_path=fll_model_221023_path,
    confidence_threshold=0.25,
    device="cuda:0",
    image_size=960
)

In [9]:
model = fll_model_221023
model_path = fll_model_221023_path

## Interact로 인퍼런스 샘플 확인

In [10]:
from sahi.utils.cv import Colors
import numpy as np
import copy

def visualize_object_predictions(
    image: np.array,
    object_prediction_list,
    rect_th: int = None,
    text_size: float = None,
    text_th: float = None,
    color: tuple = None,
):
    """
    Visualizes prediction category names, bounding boxes over the source image
    and exports it to output folder.
    Arguments:
        object_prediction_list: a list of prediction.ObjectPrediction
        rect_th: rectangle thickness
        text_size: size of the category name over box
        text_th: text thickness
        color: annotation color in the form: (0, 255, 0)
        output_dir: directory for resulting visualization to be exported
        file_name: exported file will be saved as: output_dir+file_name+".png"
        export_format: can be specified as 'jpg' or 'png'
    """
    # deepcopy image so that original is not altered
    image = copy.deepcopy(image)
    # select predefined classwise color palette if not specified
    if color is None:
        colors = Colors()
    else:
        colors = None
    # set rect_th for boxes
    rect_th = rect_th or max(round(sum(image.shape) / 2 * 0.001), 1)
    # set text_th for category names
    text_th = text_th or max(rect_th - 1, 1)
    # set text_size for category names
    text_size = text_size or rect_th / 3
    # add bbox and mask to image if present
    for object_prediction in object_prediction_list:
        # deepcopy object_prediction_list so that original is not altered
        object_prediction = object_prediction.deepcopy()

        bbox = object_prediction.bbox.to_voc_bbox()
        category_name = object_prediction.category.name
        score = object_prediction.score.value

        # set color
        if colors is not None:
            color = colors(object_prediction.category.id)
        # visualize masks if present
        if object_prediction.mask is not None:
            # deepcopy mask so that original is not altered
            mask = object_prediction.mask.bool_mask
            # draw mask
            rgb_mask = apply_color_mask(mask, color)
            image = cv2.addWeighted(image, 1, rgb_mask, 0.4, 0)
        # set bbox points
        p1, p2 = (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3]))
        # visualize boxes
        cv2.rectangle(
            image,
            p1,
            p2,
            color=color,
            thickness=rect_th
        )
        # arange bounding box text location
        label = f"{category_name} {score:.2f}"
        w, h = cv2.getTextSize(label, 0, fontScale=text_size, thickness=text_th)[0]  # label width, height
        outside = p1[1] - h - 3 >= 0  # label fits outside box
        p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
        # add bounding box text
        cv2.rectangle(image, p1, p2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(
            image,
            label,
            (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
            0,
            text_size,
            (255, 255, 255),
            thickness=text_th,
        )
        
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
    return image

In [ ]:
from ipywidgets import interact
import matplotlib.pyplot as plt
%matplotlib inline

image_files = sorted(os.listdir(source_image_dir))

@interact(index=(0, len(image_files)-1),
          slice_size=(0, 640),
          overlap_ratio=(0, 0.5, 0.05),
          custom_slice_x_start=(0, 640),
          custom_slice_y_start=(0, 512),
          custom_slice_mode=(0,3),
          only_full_inference=(0,1))
def show_sample(index=0, slice_size=640, overlap_ratio=0.25,
                custom_slice_x_start=640, custom_slice_y_start=360, custom_slice_mode=2,
                only_full_inference=0):
    image_file = image_files[index]
    image_path = os.path.join(source_image_dir, image_file)
    image = cv2.imread(image_path)
    
    if not only_full_inference:
        slice_result = slice_image(image_path, 
                                  slice_width=slice_size,
                                  slice_height=slice_size,
                                  overlap_height_ratio=overlap_ratio,
                                  overlap_width_ratio=overlap_ratio,
                                  custom_slice_x_start=custom_slice_x_start,
                                  custom_slice_y_start=custom_slice_y_start,
                                  custom_slice_mode=custom_slice_mode,
                                  verbose=1)

        for start_pixel in slice_result.starting_pixels:
            cv2.rectangle(image,
                          start_pixel,
                          [s1+s2 for s1, s2 in zip(start_pixel,[slice_size,slice_size])],
                          color=(255, 255, 0),
                          thickness=2)
        
        result = get_sliced_prediction(image_path,
                                       model,
                                       slice_height=slice_size,
                                       slice_width=slice_size,
                                       postprocess_match_threshold=0.5,
                                       overlap_height_ratio=overlap_ratio,
                                       overlap_width_ratio=overlap_ratio,
                                       custom_slice_x_start=custom_slice_x_start,
                                       custom_slice_y_start=custom_slice_y_start,
                                       custom_slice_mode=custom_slice_mode
                                      )
    else:
        result = get_prediction(image_path, model)
    
    canvas = visualize_object_predictions(image, result.object_prediction_list)
    plt.figure(figsize=(16,16))
    plt.imshow(canvas)
    plt.axis('off')
    plt.show()

In [ ]:
def initial_extract(img_dir, label_dir, out_dir):
    if os.path.exists(os.path.join(out_dir, 'val.json')):
        os.remove(os.path.join(out_dir, 'val.json'))
    
    licenses = [
        {
            "name": "",
            "id": 0,
            "url": ""
        }
    ]

    info_ = [
        {
            "contributor": "",
            "date_created": "",
            "description": "",
            "url": "",
            "version": "",
            "year": ""
        }
    ]

    categories = [
        {
            "id": 0,
            "name": "Buoy",
            "supercategory": ""
        },
        {
            "id": 1,
            "name": "Boat",
            "supercategory": ""
        },
        {
            "id": 2,
            "name": "Channel Marker",
            "supercategory": ""
        },
        {
            "id": 3,
            "name": "Speed Warning Sign",
            "supercategory": ""
        }
    ]

    img_idx = 0
    annot_idx = 0

    imgs_list = []
    annots_list = []

    for label_file in sorted(os.listdir(label_dir)):
        label_file_ = os.path.join(label_dir, label_file)
        img_file_ = os.path.join(img_dir, f'{os.path.splitext(label_file)[0]}.jpg')
        img = Image.open(img_file_)
        image_w, image_h = img.size

        imgs_list.append({
            'id': img_idx,
            'width': image_w,
            'height': image_h,
            'file_name': f'{os.path.splitext(label_file)[0]}.jpg',
            "license": 0,
            "flickr_url": "",
            "coco_url": "",
            "date_captured": 0
        })

        with open(label_file_, 'r') as label_f:
            labels = label_f.readlines()

            for label in labels:
                cat, xc, yc, label_normalized_w, label_normalized_h = list(map(lambda x: int(x) if len(x) == 1 else float(x), label.split()))
                label_w, label_h = image_w * label_normalized_w, image_h * label_normalized_h
                xmin, ymin = (image_w * xc) - (label_w / 2), (image_h * yc) - (label_h / 2)
                
                xmin = 0 if xmin < 0 else xmin
                ymin = 0 if ymin < 0 else ymin

                annots_list.append({
                    'id': annot_idx,
                    'image_id': img_idx,
                    'category_id': cat,
                    'area': int(label_h * label_w),
                    'bbox': [
                        xmin,
                        ymin,
                        label_w,
                        label_h
                    ],
                    'iscrowd': 0,
                    'attributes': {
                        'type': '',
                        'occluded': False
                    },
                    'segmentation': []
                })

                annot_idx += 1

        img_idx += 1

    out_dict = {
        'licenses': licenses,
        'info': info_,
        'categories': categories,
        'images': imgs_list,
        'annotations': annots_list
    }
    
    if not os.path.exists(out_dir):
        os.makedirs(out_dir, exist_ok=True)
    
    with open(os.path.join(out_dir, 'val.json'), 'w') as out_f:
        print(os.path.join(out_dir, 'val.json'))
        json.dump(out_dict, out_f)
        
    return os.path.join(out_dir, 'val.json')

In [ ]:
# initial_extract(img_dir, label_dir, out_dir)
gt_json_path = initial_extract(source_image_dir, source_label_dir, str(Path(source_image_dir).parent))

In [ ]:
save_val_image_dir = '../resources/save_val2/images/'
save_val_label_dir = '../resources/save_val2/labels/'

os.makedirs(save_val_image_dir, exist_ok=True)
os.makedirs(save_val_label_dir, exist_ok=True)

In [ ]:
# some handy functions to use along widgets
from IPython.display import display, Markdown, clear_output
# widget packages
import ipywidgets as widgets

from ipywidgets import interact
import matplotlib.pyplot as plt
%matplotlib inline

image_files = sorted(os.listdir(source_image_dir))
label_files = sorted([fn for fn in os.listdir(source_label_dir) if fn.endswith("txt")])

@interact(index=(0, len(image_files)-1), save=(0,1))
def show_sample(index=0, save=0):
    global button

    image_file = image_files[index]
    image_path = os.path.join(source_image_dir, image_file)
    image = cv2.imread(image_path)

    if save == 1:
        label_file = os.path.join(source_label_dir, label_files[index])
        new_label_file_name = f"VAL_{label_files[index]}"
        new_image_file_name = f"VAL_{image_files[index]}"

        print(save_val_label_dir+new_label_file_name, save_val_image_dir+new_image_file_name)
        shutil.copy(label_file, save_val_label_dir+new_label_file_name)
        shutil.copy(image_path, save_val_image_dir+new_image_file_name)
    
    result = get_prediction(image_path, model)
    
    canvas = visualize_object_predictions(image, result.object_prediction_list)
    plt.figure(figsize=(16,16))
    plt.imshow(canvas)
    plt.axis('off')
    plt.show()

In [18]:
save_val_image_dir = '../resources/save_fl221024_3/images/'
save_val_label_dir = '../resources/save_fl221024_3/labels/'
source_image_dir = "../resources/export_frame/FL_221024_3/"

os.makedirs(save_val_image_dir, exist_ok=True)
os.makedirs(save_val_label_dir, exist_ok=True)

In [19]:
# some handy functions to use along widgets
from IPython.display import display, Markdown, clear_output
# widget packages
import ipywidgets as widgets

from ipywidgets import interact
import matplotlib.pyplot as plt
%matplotlib inline

size=(1920,1080)
image_files = sorted(os.listdir(source_image_dir))
# label_files = sorted([fn for fn in os.listdir(source_label_dir) if fn.endswith("txt")])

def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

@interact(index=(0, len(image_files)-1), save=(0,1))
def show_sample(index=0, save=0):
    image_file = image_files[index]
    image_path = os.path.join(source_image_dir, image_file)
    image = cv2.imread(image_path)
    
    result = get_prediction(image_path, model)
    
    if save == 1:
        label_file_name = "FL221024_2_"+os.path.splitext(image_file)[0]+".txt"
        image_file_name = f"FL221024_2_{image_files[index]}"

        shutil.copy(image_path, save_val_image_dir+image_file_name)
    
        Path(save_val_label_dir+label_file_name).touch()
        with open(save_val_label_dir+label_file_name, 'w') as f:    
            for object_prediction in result.object_prediction_list:
                object_prediction = object_prediction.deepcopy()
                bbox = object_prediction.bbox.to_voc_bbox()
                category_id = object_prediction.category.id
                yolo = convert(size,[bbox[0], bbox[2], bbox[1], bbox[3]])
                f.write(f"{category_id} {yolo[0]} {yolo[1]} {yolo[2]} {yolo[3]}\n")
                
        print(save_val_image_dir+image_file_name, save_val_label_dir+label_file_name)

    canvas = visualize_object_predictions(image, result.object_prediction_list)
    plt.figure(figsize=(16,16))
    plt.imshow(canvas)
    plt.axis('off')
    plt.show()

interactive(children=(IntSlider(value=0, description='index', max=1037), IntSlider(value=0, description='save'…

In [ ]:
from pycocotools.coco import COCO
plt.rcParams['figure.figsize'] = (16, 8)
coco=COCO(gt_json_path)
y_offset=-20

@interact(index=(0, len(image_files)-1))
def draw_gt_bbox(index=0):
    fig, ax = plt.subplots()
    img = coco.loadImgs(ids=[index])[-1]
    I = cv2.imread(os.path.join(source_image_dir, img['file_name']))
    I = cv2.cvtColor(I, cv2.COLOR_BGR2RGB)
    ax.imshow(I); plt.axis('off')
    annIds = coco.getAnnIds(imgIds=img['id'], iscrowd=None)
    anns = coco.loadAnns(annIds)
    coco.showAnns(anns, draw_bbox=True)
    for i, ann in enumerate(anns):
        ax.text(anns[i]['bbox'][0], anns[i]['bbox'][1]+y_offset, anns[i]['category_id'], style='italic', 
                bbox={'facecolor': 'white', 'alpha': 0.7, 'pad': 3})

## remove no label images

In [ ]:
cnt = 0
for image_file in os.listdir(save_val_image_dir):
    label_file = save_val_label_dir+os.path.splitext(image_file)[0]+".txt"
    cnt += os.path.exists(label_file)
    if not os.path.exists(label_file):
        print("label file does not exit")
        os.remove(os.path.join(save_val_image_dir,image_file))
print(cnt)

In [ ]:
len(os.listdir(save_val_image_dir)) == len(os.listdir(save_val_label_dir))

## 기존 VAL에서 복사한 이미지들 삭제하기

In [ ]:
for image_name in sorted(os.listdir(save_val_image_dir)):
    label_file = save_val_label_dir+os.path.splitext(image_name)[0][4:]+".txt"
    image_file = os.path.join(save_val_image_dir,image_name[4:])
    
    os.remove(os.path.join(source_image_dir,image_name[4:]))
    os.remove(os.path.join(source_label_dir,os.path.splitext(image_name)[0][4:]+".txt"))

In [ ]:
len(os.listdir(save_val_image_dir))

In [ ]:
len(os.listdir(source_image_dir))

## YOLOv5 인퍼런스

In [ ]:
yolo_image_dir = '../resources/log_221017_2_narrow/images/'
yolo_label_dir = '../resources/log_221017_2_narrow/labels/'

In [ ]:
os.makedirs(yolo_image_dir, exist_ok=True)
os.makedirs(yolo_label_dir, exist_ok=True)

In [ ]:
from ipywidgets import interact
import matplotlib.pyplot as plt
from tqdm import tqdm

%matplotlib inline

slice_size=640
overlap_ratio=0.25
single_row_predict=True
single_row_y_start=200
postprocess_match_threshold=0.5
no_slice_prediction=True
cnt=0
size=(1920,1080)

def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

for image_file in tqdm(sorted(glob(source_image_dir+"/*.jpg"))):
    
    shutil.copy(image_file, yolo_image_dir+os.path.basename(image_file))
    label_file_name = os.path.splitext(os.path.basename(image_file))[0]+".txt"
    
    if not no_slice_prediction:
        result = get_sliced_prediction(image_file,
                                       model,
                                       slice_height=slice_size,
                                       slice_width=slice_size,
                                       postprocess_match_threshold=postprocess_match_threshold,
                                       overlap_height_ratio=overlap_ratio,
                                       overlap_width_ratio=overlap_ratio,
                                       single_row_y_start=single_row_y_start,
                                       single_row_predict=single_row_predict,
                                       verbose=0)
    else:
        result = get_prediction(image_file, model)

    Path(yolo_label_dir+label_file_name).touch()
    with open(yolo_label_dir+label_file_name, 'w') as f:    
        for object_prediction in result.object_prediction_list:
            object_prediction = object_prediction.deepcopy()
            bbox = object_prediction.bbox.to_voc_bbox()
            category_id = object_prediction.category.id
            yolo = convert(size,[bbox[0], bbox[2], bbox[1], bbox[3]])
            f.write(f"{category_id} {yolo[0]} {yolo[1]} {yolo[2]} {yolo[3]}\n")

## Visualize yolo bbox

In [ ]:
CLASS_NAME_TO_ID = {'Buoy': 0, 'Boat': 1, 'Channel Marker': 2, 'Speed Warning Sign': 3}
CLASS_ID_TO_NAME = {0: 'Buoy', 1: 'Boat', 2: 'Channel Marker', 3: 'Speed Warning Sign'}
SIZE=(1920,1080)

def visualize_yolo(image, bboxes, category_ids, color=None):
    image = copy.deepcopy(image)
    
    if color is None:
        colors = Colors()
    else:
        colors = None
        
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = CLASS_ID_TO_NAME[category_id[0]]
        
        x_center, y_center, w, h = bbox
        x_min = int((x_center - w/2) * SIZE[0])
        y_min = int((y_center - h/2) * SIZE[1])
        x_max = int((x_center + w/2) * SIZE[0])
        y_max = int((y_center + h/2) * SIZE[1])

        if colors is not None:
            color = colors(category_id[0])
        
        # set rect_th for boxes
        rect_th = max(round(sum(image.shape) / 2 * 0.001), 1)
        # set text_th for category names
        text_th = max(rect_th - 1, 1)
        # set text_size for category names
        text_size = rect_th / 3

        cv2.rectangle(image, 
                      (x_min, y_min),
                      (x_max, y_max), 
                      color=color,
                      thickness=rect_th)
        
        p1, p2 = (int(x_min), int(y_min)), (int(x_max), int(y_max))
        label = f"{class_name}"
        w, h = cv2.getTextSize(label, 0, fontScale=text_size, thickness=text_th)[0]  # label width, height
        outside = p1[1] - h - 3 >= 0  # label fits outside box
        p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
        
        cv2.rectangle(image, p1, p2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(
            image,
            label,
            (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
            0,
            text_size,
            (255, 255, 255),
            thickness=text_th,
        )
        
    return image

In [ ]:
infer_label_files = sorted(glob(yolo_label_dir+"/*.txt"))
infer_image_files = sorted(glob(yolo_image_dir+"/*.jpg"))

@interact(index=(0,len(infer_image_files)))
def verify_gt(index=0):
    image = cv2.imread(infer_image_files[index])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    label = infer_label_files[index]
    
    bboxes = []
    category_ids = []
    
    with open(label, 'r') as f:
        line = f.readline().strip()
        while line:
            id, cx, cy, w, h = list(map(float, line.split()))
            bboxes.append([cx, cy, w, h])
            category_ids.append([int(id)])
            line = f.readline().strip()

    if len(bboxes) > 0:    
        canvas = visualize_yolo(image, bboxes, category_ids)
        plt.figure(figsize=(16,16))
        plt.imshow(canvas)
        plt.axis('off')
        plt.show()